In [1]:
using Revise
using BilevelTrajOpt

In [21]:
# test the solution
function f(x)
    return x[1] * x[4] * (x[1] + x[2] + x[3]) + x[3]
end

function h(x)
    return [40. - (x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2)]
end

function g(x)
    return [25. - (x[1]*x[2]*x[3]*x[4])]
end

x0 = [1., 1., 1., 1.]
λ0 = ones(length(h(x0)))
μ0 = ones(length(g(x0)))

x_min = 1.*ones(length(x0))
x_max = 5.*ones(length(x0))

x_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g,x_min,x_max)

x_sol_known = [1.000, 4.743, 3.821, 1.379]

display(x_sol)
display(x_sol_ip)
display(x_sol_known)

4-element Array{Float64,1}:
 1.0    
 4.99846
 5.0    
 1.0    

4-element Array{Float64,1}:
 1.0    
 4.743  
 3.82115
 1.37941

4-element Array{Float64,1}:
 1.0  
 4.743
 3.821
 1.379

Solve_Succeeded


In [111]:
# test the gradient
function solve_prob(z)
    function f(x)
#         return x[1] * x[4] * (x[1] + x[2] + x[3]) + x[3]
        return dot(x,x)
    end

    function h(x)
#         return [z[1] - (x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2)]
        return z[1] - x[1] + x[4]
    end

    function g(x)
#         return [z[2] - (x[1]*x[2]*x[3]*x[4])]
        return z[2] - x[2] - x[3]*5
    end

    x0 = [1., 1., 1., 1.]
    λ0 = ones(length(h(x0)))
    μ0 = ones(length(g(x0)))

    x_min = -10.*ones(length(x0))
    x_max = 10.*ones(length(x0))

    x_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)
    
    x_sol
end

solve_prob (generic function with 2 methods)

In [112]:
z0 = [2.,-1.]

# autodiff 
J_auto = ForwardDiff.jacobian(solve_prob,z0)

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - solve_prob(z0))/ϵ
end

display(solve_prob(z0))

display(J_auto)
display(J_num)

4-element Array{Float64,1}:
  0.999893
 -0.038458
 -0.19229 
 -0.999893

4×2 Array{Float64,2}:
  0.49993  0.0     
  0.0      0.038458
  0.0      0.19229 
 -0.49993  0.0     

4×2 Array{Float64,2}:
  0.499933  0.0      
  0.0       0.0385317
  0.0       0.192291 
 -0.499933  0.0      